Ok - we have to generate a spades set ourselves for the illumina only assemblies.

First I think I need to somehow grab the spreadsheets from trish or other to organize things.

I'll make a list here about where things are (just in md) then start getting it together.

First things first - let's load trish's spreadsheet

In [1]:
import pandas as pd
import glob
import re
import os

In [2]:
workdir = '/atium/Data/nanopore/cpowgs/trees/200811_timp_tree/'
%cd /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree

/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree


In [3]:
!ls

encl	     esco	  from_trish_isolates.xlsx  klpn_spades  spades.out
encl_spades  esco_spades  klpn			    refs


In [4]:
klpn=pd.read_excel("from_trish_isolates.xlsx", sheet_name="KLPN", skiprows=3, header=None, usecols="A:F", names=["id", "cp", "cp_gene", "mlst", "repA", "year"])
encl=pd.read_excel("from_trish_isolates.xlsx", sheet_name="ENCL", skiprows=2, header=None, usecols="A:E", names=["id", "cp", "cp_gene", "repA", "year"])
esco=pd.read_excel("from_trish_isolates.xlsx", sheet_name="ESCO", skiprows=2, header=None, usecols="A:F", names=["id", "cp", "cp_gene", "mlst", "repA", "year"])

Ok - next thing to do, we need to look for the genomes
We start with pilon polished genomes, then according to yfan spades genomes/illumina data, then nanopore only assemblies.  I'm not sure if that's the right. 

In [5]:
assemblydir='/atium/Data/Nanopore/cpowgs/assemblies/'
illdirall='/atium/Data/Nanopore/cpowgs/illumina_*/'
illdir1='/atium/Data/Nanopore/cpowgs/illumina_marcc/'
illdir2='/atium/Data/Nanopore/cpowgs/illumina_2/'

In [6]:
fullill=glob.glob(illdirall+"*.fastq.gz")

In [7]:
match=[f for f in fullill if re.search(r'645[^0-9.].*R2', f)]
match

['/atium/Data/Nanopore/cpowgs/illumina_4/645KLPN_KLPN_S7_L001_R2_001.fastq.gz']

In [68]:
def assembly_find(meta, label):
    """This function finds the pilon or illumina data for the samples from a meta file"""
    meta["pilon"]=""
    meta["spades"]=""
    meta["illR1"]=""
    meta["illR2"]=""
    print(label+":")
    for i in meta.index:
        passemble=glob.glob(assemblydir+"pilon/"+"*_"+str(meta.id[i])+".pilon*")
        if passemble:
            meta.loc[i, 'pilon']=passemble
        #look for spades
        sassemble=glob.glob(label+"_spades/"+str(meta.id[i])+"/scaffolds.fasta")
        if sassemble:
            meta.loc[i, 'spades']=sassemble
        #illumina
        searchstring_r1="[^0-9S.]"+str(meta.id[i])+"[^0-9.].*R1"
        searchstring_r2="[^0-9S.]"+str(meta.id[i])+"[^0-9.].*R2"
        ill_r1=[f for f in fullill if re.search(searchstring_r1, f)]
        ill_r2=[f for f in fullill if re.search(searchstring_r2, f)]
        if ill_r1:
            #print(ill)
            meta.loc[i, 'illR1']=ill_r1[0]
            meta.loc[i, 'illR2']=ill_r2[0]
            #print("Yes Illumina in dir1")
    return(meta)

In [77]:
def spades_assemble(meta, label, isolate=True):
    for i in meta.index:
        if not meta.pilon[i]:
            if not meta.spades[i]:
                if meta.illR1[i]:
                    #let's run spades!
                    #First trim trimmomatic
                    !trimmomatic PE -quiet -threads 11 {meta.illR1[i]} {meta.illR2[i]} /tmp/read1.trim.fq.gz /tmp/read2.unpaired.fq.gz \
                        /tmp/read2.trim.fq.gz /tmp/read2.unpaired.fq.gz \
                        ILLUMINACLIP:/home/timp/miniconda3/envs/cpowgs/share/trimmomatic/adapters/NexteraPE-PE.fa:2:30:10 \
                        LEADING:10 TRAILING:10 SLIDINGWINDOW:4:20 MINLEN:40
                    #then run spades
                    if isolate: 
                        !spades.py -t 8 --isolate -1 /tmp/read1.trim.fq.gz -2 /tmp/read2.trim.fq.gz -o {label}_spades/{meta.id[i]} >{meta.id[i]}.spades.out
                    else:
                        !spades.py -t 8 -1 /tmp/read1.trim.fq.gz -2 /tmp/read2.trim.fq.gz -o {label}_spades/{meta.id[i]} >{meta.id[i]}.spades.out

In [93]:
klpn=assembly_find(klpn, "klpn")
esco=assembly_find(esco, "esco")
encl=assembly_find(encl, "encl")


klpn:
esco:
encl:


In [94]:
encl.loc[encl.id==53, 'illR1']="/atium/Data/Nanopore/cpowgs/illumina_3/53ENCL_ENCL_S81_L002_R1_001.fastq.gz"
encl.loc[encl.id==53, 'illR2']="/atium/Data/Nanopore/cpowgs/illumina_3/53ENCL_ENCL_S81_L002_R2_001.fastq.gz"
encl.loc[encl.id==3, 'illR1']="/atium/Data/Nanopore/cpowgs/illumina_3/3ENCL_ENCL_S71_L002_R1_001.fastq.gz"
encl.loc[encl.id==3, 'illR2']="/atium/Data/Nanopore/cpowgs/illumina_3/3ENCL_ENCL_S71_L002_R2_001.fastq.gz"
encl.loc[encl.id==2, 'illR1']=""
encl.loc[encl.id==2, 'illR2']=""

Ok - now what I have to do is dump the correct versions of each (pilon or spades) into a new folder

!mkdir esco_spades
!mkdir klpn_spades
!mkdir encl_spades

In [95]:
display(klpn[(klpn.pilon=='') & (klpn.spades=='')])
display(encl[(encl.pilon=='') & (encl.spades=='')])
display(esco[(esco.pilon=='') & (esco.spades=='')])

,id,cp,cp_gene,mlst,repA,year,pilon,spades,illR1,illR2
11,38,Non-CP,NaN,NaN,NaN,2016,,,,


,id,cp,cp_gene,repA,year,pilon,spades,illR1,illR2
0,1,Non-CP,NaN,NaN,2016,,,,
1,2,Non-CP,NaN,NaN,2016,,,,


,id,cp,cp_gene,mlst,repA,year,pilon,spades,illR1,illR2


Spot fixing terrible naming

In [96]:
klpn.to_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/klpn.csv")
esco.to_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/esco.csv")
encl.to_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/encl.csv")


In [97]:
spades_assemble(klpn, "klpn")
spades_assemble(esco, "esco")
spades_assemble(encl, "encl")

Ok - spades now done.  Let's take this and now start my normal tree stuff in the next notebook